In [1]:
import imagej
from scyjava import jimport
from scipy import stats
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scyjava import jimport, config
import xlsxwriter
from openpyxl import Workbook
from openpyxl import load_workbook
from glob import glob
import os
import re

In [2]:
if 'ij' in locals() and ij is not None:
    ij.dispose()
plugins_dir = r'C:\Program Files\Fiji.app\plugins'
config.add_option(f'-Dplugins.dir={plugins_dir}')
ij = imagej.init(r"C:\Program Files\Fiji.app", mode=imagej.Mode.INTERACTIVE)
print(ij.getVersion())
ij.ui().showUI()

#import imageJ
ImagePlus=jimport('ij.ImagePlus')
IJ = jimport('ij.IJ')
FileSaver = jimport('ij.io.FileSaver')
WaitForUserDialog = jimport('ij.gui.WaitForUserDialog')
ChannelSplitter = jimport('ij.plugin.ChannelSplitter')
ImageCalculator = jimport('ij.plugin.ImageCalculator')
NonBlockingGenericDialog = jimport('ij.gui.NonBlockingGenericDialog')
PointRoi = jimport('ij.gui.PointRoi')
GenericDialog = jimport('ij.gui.GenericDialog')
RoiManager = jimport('ij.plugin.frame.RoiManager')
WindowManager = jimport('ij.WindowManager')
Duplicator = jimport('ij.plugin.Duplicator')
ResultsTable = jimport('ij.measure.ResultsTable')
ImageStack = jimport('ij.ImageStack')
GaussianBlur = jimport('ij.plugin.filter.GaussianBlur')

2.14.0/1.54f


In [ ]:
path = r"D:\PhD Pauline\µmanager\231110 TDP67 - Correlation between phenotypes repartition and DNA qtt\Lamelle 2 - 647 ATXN2L, 594 p62 - 5Fm-emGFP-TDP43 500ng"
files_paths = glob(os.path.join(path, "**", "*.tif"), recursive=True)
dre = re.compile(r'(\d+)')
files_paths.sort(key=lambda l: [int(s) if s.isdigit() else s.lower() for s in re.split(dre, l)])
DNA_qtt = 2000
# Definition of output
e=0
df_data_size = pd.DataFrame(columns=['Expmt', 'DNA_qtt', 'Cell index', 'Phenotype', 'Nb of cond./cell', 'Mean Area of Cond. (µm2)', 'Mean Fluo of Cond.', 'Stressed?', 'p62?'])
df_data_phenotypes = pd.DataFrame(columns=['expmt', 'DNA_qtt', 'Nb of transfected cells', 'Monster', 'Large', 'Dot', 'Nuclear'])
IJ.run("Channels Tool...")
for filepath in files_paths:
    rm=RoiManager.getRoiManager()
    rm.reset()
    # Define savefile_path
    e += 1
    savefile_path = os.path.join(path, f'_{e}', 'Quantif_phenotype')
    os.makedirs(savefile_path, exist_ok=True)

    # Open image
    imp = IJ.openImage(filepath)
    imp.show()
    d = Duplicator()
    if (imp.getDimensions()[3]>1):
        IJ.run("Z Project...", "projection=[Max Intensity]")
        imp.close()
        imp.show()
        imp= IJ.getImage()

    # Reset contrast
    imp.setC(1)
    IJ.resetMinAndMax(imp);
    IJ.run(imp, "Enhance Contrast", "saturated=0.35");
    IJ.run(imp, "Next Slice [>]", "")
    IJ.resetMinAndMax(imp);
    IJ.run(imp, "Next Slice [>]", "")
    IJ.resetMinAndMax(imp);
    IJ.run(imp, "Enhance Contrast", "saturated=0.35");
    IJ.run(imp, "Next Slice [>]", "")
    IJ.resetMinAndMax(imp);
    imp.setC(1)

    # Reference expmt
    gui=NonBlockingGenericDialog("Experiment?")
    gui.addMessage("Ref+Lamelle")
    gui.addStringField("n°",f'TDP67_2_{e}')
    gui.enableYesNoCancel('ok', 'skip')
    gui.showDialog()

    if gui.wasCanceled():
        imp.close()
        break
    if not gui.wasOKed():
        imp.close()
        continue
    expmt=ij.py.from_java(gui.getNextString())

    # Adjust brighness and contrast
    IJ.run("Brightness/Contrast...")
    gui=NonBlockingGenericDialog("Adjust Brightness/Contrast")
    gui.enableYesNoCancel('ok', 'skip')
    gui.showDialog()

    # Select transfected cells
    # Open pre-existing ROIs
    if os.path.isfile(savefile_path + "\\" + "roi.zip"):
        rm.open(savefile_path + "\\" + "roi.zip")
        transfected_cells=rm.getRoisAsArray()
        transfected = len(transfected_cells)
    else:
        IJ.setTool("freehand")
        imp.setC(3)
        WaitForUserDialog("Select the different transfected Cells").show()
        transfected_cells=rm.getRoisAsArray()
        rm.runCommand(imp,"Show All");

        if not transfected_cells:
            WaitForUserDialog("Empty ROI Manager. Press 't' key after selecting cells. One more chance:").show()
            transfected_cells=rm.getRoisAsArray()

        transfected = len(transfected_cells)
        rm.save(savefile_path + "\\" + "roi.zip")


    #loop for each cell with granules. Allows to input Normal/Abnormal phenotype for each specific granule
        # transfected_cells = table of ROIs, cell_roi = roi
    nb_large = 0
    nb_monster = 0
    nb_dot = 0
    nb_nuclear = 0
    index = 0
    data_phenotype_imp =[]
    data_size_imp = []

    for cell_roi in transfected_cells:
        index += 1
        crop = imp.crop([cell_roi])[0]
        crop.show()
        crop.setC(3)
        crop.setActiveChannels("1010")

        gui=NonBlockingGenericDialog("Phenotype?")
        gui.addChoice("Choose", ["dot","large", "monster", "nuclear"], "Skip")
        gui.showDialog()
        inChoice=gui.getNextChoice()
        large = 'large'
        monster = 'monster'
        dot = "dot"
        nuclear = "nuclear"

        if inChoice == nuclear:
            nb_nuclear +=1
            crop.close()
            continue

        else:
            # Cell with SGs ?
            gui=NonBlockingGenericDialog("Stress granules?")
            gui.addChoice("Choose", ["no", "yes"], "Skip")
            gui.showDialog()
            SG_choice=gui.getNextChoice()
            yes = 'yes'
            no = 'no'

            gui=NonBlockingGenericDialog("p62?")
            gui.addChoice("Choose", ["non", "oui"], "Skip")
            gui.showDialog()
            p62_choice=gui.getNextChoice()
            oui = 'oui'
            non = 'non'

            if inChoice == large:
                nb_large += 1


            elif inChoice==monster:
                nb_monster +=1


            # Number and mean size of condensates
            # Create a mask around the selected cell
            splitted = ChannelSplitter.split(crop)
            crop_GFP = splitted[2]
            cell_mask = ImagePlus('cell_mask', cell_roi.getMask())
            IJ.run(cell_mask, "Divide...", "value=255.000")
            #Reduce background (only for dotted pattern)
            cell = ImageCalculator.run(crop_GFP, cell_mask, 'multiply create')
            cell_blur = cell.duplicate()
            if inChoice == dot:
                nb_dot +=1
                GaussianBlur().blurGaussian(cell_blur.getProcessor(),2)
                deblured_cell = ImageCalculator.run(cell, cell_blur, "Subtract create")
            else:
                deblured_cell = cell_blur
            crop.close()
            deblured_cell.show()
            # Apply a threshold and analyze particles > 70nm:
            IJ.run("Threshold...")
            IJ.setAutoThreshold(deblured_cell, "Otsu dark no-reset")
            gui=NonBlockingGenericDialog("Set threshold green?")
            gui.addMessage("Threshold of condensates")
            gui.showDialog()
            IJ.run("Set Measurements...", "area mean redirect=None decimal=3");
            IJ.run(deblured_cell, "Analyze Particles...", "size=0.07-Infinity display clear add")
            ResultsTable=jimport('ij.measure.ResultsTable')
            rt = ResultsTable.getResultsTable()

            # Count condensates per cell and extract their mean size and mean intensity
            results_area_java = rt.getColumnAsDoubles(0)
            results_mean_java = rt.getColumnAsDoubles(1)
            results_area = np.array(results_area_java)
            results_meanfluo = np.array(results_mean_java) #convert java object to np array
            mean_area = results_area.mean()
            nb_cond = results_area.shape[0]
            mean_fluo = results_meanfluo.mean()


        # Results for 1 cell size&fluo
        results_size = [expmt, DNA_qtt, index, inChoice, nb_cond, mean_area, mean_fluo, SG_choice, p62_choice]
        data_size_imp += [results_size]

        rm.reset()
        deblured_cell.close()

    # Results for phenotypes
    results_phenotype = [expmt, DNA_qtt, transfected, nb_monster, nb_large, nb_dot, nb_nuclear]
    data_phenotype_imp += [results_phenotype]

    # Stockage data for size & fluo
    df_size = pd.DataFrame(data_size_imp, columns=['Expmt', 'DNA_qtt', 'Cell index', 'Phenotype', 'Nb of cond./cell', 'Mean Area of Cond. (µm2)', 'Mean Fluo of Cond.', 'Stressed?', 'p62?'])
    df_data_size = pd.concat([df_data_size, df_size], ignore_index=True)

    # Stockage data for phenotypes
    df_phenotypes = pd.DataFrame(data_phenotype_imp, columns=['expmt', 'DNA_qtt', 'Nb of transfected cells', 'Monster', 'Large', 'Dot', 'Nuclear'])
    df_data_phenotypes = pd.concat([df_data_phenotypes, df_phenotypes], ignore_index=True)

    display(df_data_phenotypes)
    display(df_data_size)

    imp.close()
display(df_data_phenotypes)
display(df_data_size)

In [61]:
# Saving data to excel
df_data_size.to_excel('D:\PhD Pauline\Data\Phenotypes_size\Size_raw.xlsx', index=False)
df_data_phenotypes.to_excel('D:\PhD Pauline\Data\Phenotypes_size\Phenotypes_raw.xlsx', index=False)

#Read raw data from excel
raw_size = pd.read_excel('D:\PhD Pauline\Phenotypes_size\Size_raw.xlsx')
raw_phenotypes = pd.read_excel('D:\PhD Pauline\Phenotypes_size\Phenotypes_raw.xlsx')

#Read all data from excel
data_size = pd.read_excel('D:\PhD Pauline\Phenotypes_size\Size.xlsx')
data_phenotypes = pd.read_excel('D:\PhD Pauline\Phenotypes_size\Phenotypes.xlsx')

#Append via pd.concat()
data_size =pd.concat([data_size, raw_size])
data_size.to_excel('D:\PhD Pauline\Phenotypes_size\Size.xlsx')
data_phenotypes = pd.concat([data_phenotypes, raw_phenotypes])
data_phenotypes.to_excel('D:\PhD Pauline\Phenotypes_size\Phenotypes.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\PhD Pauline\\Phenotypes_size\\Size_raw.xlsx'

In [62]:
sheet_name = '500ng'

# Saving data to excel
df_data_size.to_excel('D:\PhD Pauline\Data\Phenotypes_size\Size_raw.xlsx', index=False)
df_data_phenotypes.to_excel('D:\PhD Pauline\Data\Phenotypes_size\Phenotypes_raw.xlsx', index=False)

#Read raw data from excel
raw_size = pd.read_excel("D:\PhD Pauline\Data\Phenotypes_size\Size_raw.xlsx")
raw_phenotypes = pd.read_excel("D:\PhD Pauline\Data\Phenotypes_size\Phenotypes_raw.xlsx")

#Read all data from excel
data_size = pd.read_excel('D:\PhD Pauline\Data\Phenotypes_size\Size.xlsx')
data_phenotypes = pd.read_excel("D:\PhD Pauline\Data\Phenotypes_size\Phenotypes.xlsx")

#Append via pd.concat()
data_size =pd.concat([data_size, raw_size])
data_size.to_excel("D:\PhD Pauline\Data\Phenotypes_size\Size.xlsx", sheet_name=sheet_name, index=False)
data_phenotypes = pd.concat([data_phenotypes, raw_phenotypes])
data_phenotypes.to_excel("D:\PhD Pauline\Data\Phenotypes_size\Phenotypes.xlsx", sheet_name=sheet_name, index = False)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\PhD Pauline\\Data\\Phenotypes_size\\Size.xlsx'

In [11]:
# Saving data to excel
df_data_size.to_excel('D:\PhD Pauline\Data\Phenotypes_size\Size_raw.xlsx', index=False)
df_data_phenotypes.to_excel('D:\PhD Pauline\Data\Phenotypes_size\Phenotypes_raw.xlsx', index=False)